In [1]:
DATA_NAME = 'pastis-full' 
TRANSFORM = 'fourier'
CHANNEL = 'blue'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_blue_fourier_full_pastis.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
        cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,466.06,-0.03,0.01,-0.01,-0.01,0.01,0.01,-0.01,0.00,-0.00,...,0.0,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,0.0,-0.0
3,-0.03,348.20,0.00,0.01,0.01,0.01,-0.00,-0.01,0.01,-0.00,...,-0.0,-0.00,-0.00,-0.00,-0.00,0.00,0.00,0.00,-0.0,-0.0
4,0.01,0.00,288.65,0.02,-0.01,-0.00,0.00,-0.00,0.00,0.00,...,0.0,-0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.0,0.0
5,-0.01,0.01,0.02,222.79,0.01,-0.00,0.00,0.01,0.00,0.00,...,0.0,0.00,-0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.0,0.0
6,-0.01,0.01,-0.01,0.01,174.15,0.00,0.00,-0.00,-0.00,-0.00,...,-0.0,0.00,-0.00,-0.00,0.00,-0.00,-0.00,0.00,0.0,0.0
7,0.01,0.01,-0.00,-0.00,0.00,132.99,0.00,-0.00,0.00,0.00,...,-0.0,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.0,0.0
8,0.01,-0.00,0.00,0.00,0.00,0.00,105.90,0.01,0.00,-0.00,...,-0.0,0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,-0.0,-0.0
9,-0.01,-0.01,-0.00,0.01,-0.00,-0.00,0.01,83.94,-0.01,0.00,...,0.0,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.0,-0.0
10,0.00,0.01,0.00,0.00,-0.00,0.00,0.00,-0.01,66.03,0.00,...,0.0,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.0,0.0
11,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,51.59,...,0.0,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.0,-0.0


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10,11,...,24,25,26,27,28,29,30,31,32,33
2,1.00000,-0.00007,0.00002,-0.00003,-0.00002,0.00003,0.00003,-0.00003,0.00002,-0.00000,...,0.00002,0.00001,0.00002,-0.00002,-0.00000,-0.00000,0.00002,0.00001,0.00002,-0.00000
3,-0.00007,1.00000,0.00000,0.00002,0.00003,0.00005,-0.00002,-0.00003,0.00003,-0.00000,...,-0.00002,-0.00002,-0.00002,-0.00001,-0.00001,0.00000,0.00000,0.00001,-0.00000,-0.00001
4,0.00002,0.00000,1.00000,0.00006,-0.00003,-0.00001,0.00002,-0.00001,0.00001,0.00000,...,0.00000,-0.00000,-0.00001,-0.00002,0.00001,-0.00001,-0.00000,0.00001,0.00000,0.00001
5,-0.00003,0.00002,0.00006,1.00000,0.00004,-0.00002,0.00002,0.00005,0.00001,0.00000,...,0.00001,0.00002,-0.00000,-0.00002,-0.00002,0.00001,-0.00000,0.00001,-0.00000,0.00000
6,-0.00002,0.00003,-0.00003,0.00004,1.00000,0.00000,0.00003,-0.00001,-0.00002,-0.00000,...,-0.00000,0.00001,-0.00000,-0.00001,0.00001,-0.00000,-0.00000,0.00001,0.00000,0.00000
7,0.00003,0.00005,-0.00001,-0.00002,0.00000,1.00000,0.00002,-0.00000,0.00001,0.00000,...,-0.00004,-0.00000,-0.00000,-0.00001,0.00002,0.00001,-0.00003,-0.00002,-0.00001,0.00002
8,0.00003,-0.00002,0.00002,0.00002,0.00003,0.00002,1.00000,0.00007,0.00004,-0.00000,...,-0.00003,0.00000,0.00001,0.00000,-0.00003,-0.00001,-0.00000,0.00000,-0.00001,-0.00001
9,-0.00003,-0.00003,-0.00001,0.00005,-0.00001,-0.00000,0.00007,1.00000,-0.00007,0.00001,...,0.00001,-0.00004,-0.00000,-0.00001,-0.00002,-0.00001,-0.00000,0.00000,0.00001,-0.00000
10,0.00002,0.00003,0.00001,0.00001,-0.00002,0.00001,0.00004,-0.00007,1.00000,0.00005,...,0.00000,-0.00000,0.00000,-0.00003,0.00001,-0.00000,0.00001,-0.00000,0.00000,0.00000
11,-0.00000,-0.00000,0.00000,0.00000,-0.00000,0.00000,-0.00000,0.00001,0.00005,1.00000,...,0.00001,-0.00002,0.00001,-0.00001,-0.00002,0.00001,-0.00002,0.00000,-0.00001,-0.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.0645768933410333

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[6.80510131e+03 3.79526864e+03 2.59900362e+03 1.54728643e+03
 9.43961246e+02 5.50180173e+02 3.48180424e+02 2.18366949e+02
 1.34865670e+02 8.21948806e+01 4.95226392e+01 3.25136148e+01
 1.70399287e+01 1.08467019e+01 6.21164074e+00 3.49564171e+00
 1.92200910e+00 1.03052710e+00 5.34467124e-01 2.90378974e-01
 1.39093198e-01 6.46696015e-02 2.90903912e-02 1.30143651e-02
 5.12712874e-03 1.89745530e-03 6.78402612e-04 2.09061144e-04
 1.67042819e-05 2.18537024e-06 1.67185788e-07 1.34206365e-27]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.997815,-0.052223,-0.030252,-0.018537,-0.012935,-0.009192,-0.007082,-0.005531,-0.004271,-0.003319,...,-0.000062,-0.000042,-0.000026,-0.000017,-0.000010,-0.000006,-0.000001,-5.937351e-07,-1.557552e-07,-5.384648e-08
1,0.048223,0.994114,-0.086204,-0.033562,-0.020409,-0.013589,-0.010220,-0.007805,-0.005959,-0.004603,...,-0.000088,-0.000060,-0.000038,-0.000023,-0.000014,-0.000008,-0.000002,-8.328081e-07,-2.873529e-07,-8.645338e-08
2,0.032220,0.080944,0.992896,-0.067836,-0.032828,-0.019998,-0.014423,-0.010806,-0.008180,-0.006261,...,-0.000117,-0.000079,-0.000050,-0.000031,-0.000018,-0.000011,-0.000003,-1.094160e-06,-3.601011e-07,-8.132186e-08
3,0.020538,0.035342,0.060551,0.993589,-0.073358,-0.032731,-0.021430,-0.015227,-0.011266,-0.008478,...,-0.000154,-0.000104,-0.000066,-0.000041,-0.000025,-0.000014,-0.000004,-1.470212e-06,-5.052080e-07,-1.212679e-07
4,0.015110,0.022989,0.032522,0.066375,0.993233,-0.069869,-0.035641,-0.023004,-0.016147,-0.011800,...,-0.000207,-0.000139,-0.000089,-0.000055,-0.000033,-0.000019,-0.000005,-2.021440e-06,-6.375219e-07,-1.647069e-07
5,0.010917,0.015645,0.020474,0.032156,0.061625,0.992153,-0.085070,-0.039415,-0.024421,-0.016833,...,-0.000278,-0.000185,-0.000118,-0.000072,-0.000043,-0.000025,-0.000008,-2.910622e-06,-9.074194e-07,-1.802962e-07
6,0.008883,0.012441,0.015678,0.022598,0.035099,0.076008,0.990620,-0.088031,-0.040377,-0.025024,...,-0.000369,-0.000246,-0.000155,-0.000095,-0.000059,-0.000033,-0.000010,-3.641386e-06,-1.232597e-06,-3.058924e-07
7,0.007259,0.009967,0.012336,0.016947,0.024186,0.039843,0.077631,0.989995,-0.089941,-0.041054,...,-0.000483,-0.000329,-0.000206,-0.000126,-0.000078,-0.000044,-0.000013,-4.823171e-06,-1.487437e-06,-3.879373e-07
8,0.005862,0.007966,0.009780,0.013147,0.017895,0.026420,0.040314,0.078909,0.989590,-0.090913,...,-0.000642,-0.000433,-0.000273,-0.000170,-0.000101,-0.000059,-0.000017,-6.473472e-06,-2.023976e-06,-5.004981e-07
9,0.004732,0.006395,0.007787,0.010310,0.013667,0.019142,0.026621,0.040669,0.079237,0.989089,...,-0.000854,-0.000579,-0.000364,-0.000224,-0.000137,-0.000077,-0.000023,-8.526934e-06,-2.786659e-06,-6.938562e-07


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.00218458363535734,
 0.00588603992922021,
 0.007103904881877621,
 0.006411213214468803,
 0.006767119049868486,
 0.007847473839809216,
 0.009380055831922252,
 0.010005000919135587,
 0.010410260236397906,
 0.01091118725164375,
 0.013397303326281196,
 0.013158367479257804,
 0.012954016777435351,
 0.015212628390855465,
 0.014096744302047659,
 0.014460550339862599,
 0.014977627033649576,
 0.01558415149635839,
 0.01774995529902923,
 0.018708992146278858,
 0.017712210898599423,
 0.018929447270410016,
 0.021368833787299812,
 0.022878255527937652,
 0.023502053314336413,
 0.027215338920102305,
 0.03135692142428603,
 0.022972440469727462,
 0.015814652380400362,
 0.026514267703777294,
 0.04666419464063343,
 0.03348682775096046]